<a href="https://colab.research.google.com/github/basturkme/4IER-HMI/blob/main/dataLogger.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import serial
import csv
import time
import sys
import os
from datetime import datetime

# Configuration
# -------------------------------------------------------------------------
# UPDATE THESE TO MATCH YOUR HARDWARE
SERIAL_PORT = 'COM3'       # Windows: 'COM3', Linux/Mac: '/dev/ttyUSB0'
BAUD_RATE = 115200         # Must match the baud rate set in your STM32/ESP32 firmware
OUTPUT_FILENAME = 'emg_data_log.csv'
# -------------------------------------------------------------------------

def connect_serial(port, baud):
    """Establishes a connection to the microcontroller."""
    try:
        ser = serial.Serial(port, baud, timeout=1)
        # Flush input buffer to clear old data
        ser.reset_input_buffer()
        print(f"✅ Connected to {port} at {baud} baud.")
        return ser
    except serial.SerialException as e:
        print(f"❌ Error connecting to port {port}: {e}")
        sys.exit(1)

def main():
    ser = connect_serial(SERIAL_PORT, BAUD_RATE)

    # Check if file exists to handle header writing
    file_exists = os.path.isfile(OUTPUT_FILENAME)

    print(f"📡 Starting data logging to '{OUTPUT_FILENAME}'...")
    print("Press Ctrl+C to stop logging.\n")

    try:
        with open(OUTPUT_FILENAME, mode='a', newline='') as file:
            writer = csv.writer(file)

            # Write header if file is new
            if not file_exists:
                # Adjust columns based on what your MCU sends.
                # Example: Timestamp, Sensor_Value
                writer.writerow(['Timestamp_Host', 'ADC_Value_Raw'])

            while True:
                if ser.in_waiting > 0:
                    try:
                        # Read a line from the serial port (decode bytes to string)
                        line = ser.readline().decode('utf-8').strip()

                        # Filter out empty lines
                        if not line:
                            continue

                        # Generate host-side timestamp
                        timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S.%f')

                        # Print to console for monitoring
                        print(f"[{timestamp}] Received: {line}")

                        # Write to CSV
                        # Assuming the MCU sends just the number (e.g., "4095")
                        # If MCU sends comma-separated data (e.g., "4095, 2048"),
                        # you can do: data = line.split(',')
                        writer.writerow([timestamp, line])

                    except UnicodeDecodeError:
                        print("⚠️  Decoding error: Garbage data received.")
                    except ValueError:
                        print(f"⚠️  Parsing error: Could not parse '{line}'")

    except KeyboardInterrupt:
        print("\n🛑 Logging stopped by user.")
    except Exception as e:
        print(f"\n❌ Unexpected error: {e}")
    finally:
        if ser.is_open:
            ser.close()
            print("🔌 Serial connection closed.")

if __name__ == "__main__":
    main()

ModuleNotFoundError: No module named 'serial'